In [ ]:
%matplotlib widget
import pylab as plt
from labdata.schema import *

plt.matplotlib.rcParams['pdf.fonttype'] = 42
plt.matplotlib.rcParams['ps.fonttype'] = 42
savepath = Path('../../figures/figure1')


In [ ]:
UnitCount.populate(processes = 12, display_progress = True)

In [ ]:
subject = 'JC131'
q_task = (Dataset() & f'subject_name = "{subject}"' & 'dataset_name LIKE "%Droplets%"').fetch('subject_name','session_name',as_dict = True)

rec_task = pd.DataFrame((Session*UnitCount*EphysRecording.ProbeSetting & q_task &
     "parameter_set_num = 5" & 'unit_criteria_id = 1' & 'configuration_id = 3').fetch())
rec_all = pd.DataFrame((Session*UnitCount*EphysRecording.ProbeSetting & f'subject_name = "{subject}"' &
     "parameter_set_num = 5" & 'unit_criteria_id = 1' & 'configuration_id = 3').fetch())

implant_date = np.array((Procedure() & f'subject_name = "{subject}"' & 'procedure_type = "chronic implant"').fetch('procedure_datetime'),dtype = 'datetime64')


In [ ]:
import pylab as plt
fig = plt.figure(figsize = [7.48,1.35])
fig.add_axes([0.2,0.2,0.7,0.7])
days = (rec_all.session_datetime.values - implant_date).astype('timedelta64[D]').astype(int)
plt.plot(days,
         rec_all.sua,'ko',markerfacecolor = 'gray',ms = 2,markeredgewidth = 0)
days = (rec_task.session_datetime.values - implant_date).astype('timedelta64[D]').astype(int)
plt.plot(days,
         rec_task.sua,'ko',markerfacecolor = 'none',ms = 2,markeredgewidth = 0.5)

days = (rec_all.session_datetime.values - implant_date).astype('timedelta64[D]').astype(int)
plt.plot(days,
         rec_all.mua,'ko',markerfacecolor = 'none',ms = 1,markeredgewidth = 0.5)

plt.hlines(np.mean(rec_all.sua.values),days.min(),days.max(),lw = 0.5,color = 'k')

plt.ylim([0,600])
plt.xlim([0,days.max()*1.01])

plt.xticks(np.arange(0,days.max(),60),fontsize = 6)
plt.yticks(np.arange(0,550,150),fontsize = 6)
plt.xlabel('Days from implant',fontsize = 8)
plt.ylabel('# units',fontsize = 8)
plt.gca().spines[['right', 'top']].set_visible(False)
plt.text(60,200,'single units',fontsize = 7)
plt.text(180,450,'sorted units',fontsize = 7);
if not savepath.exists():
    savepath.mkdir(parents = True)
fig.savefig(savepath/f'{subject}_units_longitudinal.pdf')